## SESIÓN 2.1: Preprocesamiento y extracción de características.

### Realizado por Álvaro Mazcuñán y Miquel Marín

Se importan las librerías que se utilizarán a lo largo del ejercicio:

In [6]:
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec

import re
import string
import numpy as np

Se lee el CSV con los datos de DETOXIS.

También se crean dos funciones claves. La primera, "read_data", obtiene a partir de los datos del CSV, las variables que necesitaremos para el análisis.

La segunda de ellas, "preprocess_tweet_text", realiza el preprocesado correspondiente al primer apartdado de esta tarea. Su "output" NO está tokenizado. Esto es así debido a que 3 de las 4 funciones que se emplean más tardes requieren que el corpus no esté tokenizado. Cuando se utilice "Word2Vec" ya se modificará la función para tener los datos tokenizados.

In [7]:
data = pd.read_csv('/Users/mazcu/Documents/UPV/3º/2º Cuatrimestre/LNR/DATASET_DETOXIS.csv')

def read_data(data):
    text = list(data['comment'])
    t1_label = list(data['toxicity'])
    t2_label = list(data['toxicity_level'])
    return text, t1_label, t2_label

def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    # REMOVE URL
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE)
    # REMOVE @ AND #
    tweet = re.sub(r"\@\w+|\#", "", tweet)
    # REMOVE EMOJIS AND EMOTICONES
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet)
    # REMOVE PUNCTUATIONS
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    #REMOVE STOPWORDS
    tweet_tokens = word_tokenize(tweet)
    filtered = [w for w in tweet_tokens if not w in set(stopwords.words('spanish'))]
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    return " ".join(lemma_words) #NO TOKENIZADO
    
texts, t1_label, t2_label = read_data(data)
tweets_cleaned = [preprocess_tweet_text(tweet) for tweet in texts]

"tweets_cleaned" es una lista que contiene cada tweet ya preprocesado.




Ahora, se pasa a la parte de extracción de características:

La primera de las diferentes opciones que se van a implementar es la creación de una Bolsa de Palabras o "Bag-of-Words"

In [8]:
vectorizer = CountVectorizer()
vectorizer.fit(tweets_cleaned)

X_bag_of_words = vectorizer.transform(tweets_cleaned)

No se muestra el contenido de "X_bag_of_words" debido a que es una matriz de 3463 filas por 12700 columnas.

Si se desea conocer qué palabras se encuentran en el vocabulario, basta con ejecutar la siguiente línea: vectorizer.get_feature_names()

Aquí se muestra como queda con solo 10 palabras.


In [9]:
vectorizer.get_feature_names()[1000:1010]

['aprendemo',
 'aprenden',
 'aprendida',
 'aprendido',
 'aprendieran',
 'aprendái',
 'aprendí',
 'aprietan',
 'aprobado',
 'apropiación']

A continuación, se realizará la tarea por trigramas, que son N-Gramas de 3 palabras.

In [10]:
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(3, 3))
counts = ngram_vectorizer.fit_transform(tweets_cleaned)
counts.toarray().astype(int)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Para el TF-IDF, se implementa el siguiente código:

In [11]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(tweets_cleaned)


TfidfVectorizer()

Si se desea la importancia de las palabras, basta con ejecutar la siguiente línea: tfidf_vectorizer.vocabulary_

Aquí se muestra para las 10 primeras.

In [12]:
iter_ = 0
for word in tfidf_vectorizer.vocabulary_.keys():
    if iter_ < 10:
        print(word, tfidf_vectorizer.vocabulary_[word])        
    else:
        break
    iter_ += 1

pensó 9003
zumo 12654
restar 10533
gusta 5821
afeitado 489
seco 10902
gent 5666
asi 1169
maten 7609
alta 714


Por último, se pasa a "Words Embeddings", con "Word2Vec".

Se modifica la función de antes para tener tokenizadas las paalabras de cada tweet.

In [13]:
def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    # REMOVE URL
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE)
    # REMOVE @ AND #
    tweet = re.sub(r"\@\w+|\#", "", tweet)
    # REMOVE EMOJIS AND EMOTICONES
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet)
    # REMOVE PUNCTUATIONS
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    #REMOVE STOPWORDS
    tweet_tokens = word_tokenize(tweet)
    filtered = [w for w in tweet_tokens if not w in set(stopwords.words('spanish'))]
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    return lemma_words #TOKENIZADO
    
texts, t1_label, t2_label = read_data(data)
tweets_cleaned = [preprocess_tweet_text(tweet) for tweet in texts]

In [14]:
model = Word2Vec(
        tweets_cleaned,
        vector_size=30,
        min_count=5)

Se probará "model" para que busque por las palabras de mayor similitud con "política".

In [15]:
model.wv.most_similar('política')

[('vox', 0.9992325305938721),
 ('part', 0.9991074800491333),
 ('idea', 0.9990335702896118),
 ('creo', 0.9990204572677612),
 ('trabajo', 0.9990178942680359),
 ('millon', 0.9989981651306152),
 ('demá', 0.9988898038864136),
 ('mismo', 0.9988840222358704),
 ('muerto', 0.9988511204719543),
 ('podemo', 0.9988441467285156)]